<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_ATH_TTM_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

mypf = mypf[mypf['LatestQtr'] == 0]
#mypf = mypf[mypf['v80'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['ACC','AMBUJACEM','CADILAHC','GILLETTE','NESTLEIND','PGHH','SIEMENS', 'VALIANTORG','VINATIORGA', 'TATAELXSI', 'AMARAJABAT']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100
#stock_ids = ['DIVISLAB', 'HDFCBANK']
print('Stocks to be scanned:',len(stock_ids))

Mounted at /content/drive
Stocks to be scanned: 10


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df_latest_analysis = pd.DataFrame()

for stock_name in stock_ids: 


    print(stock_name)

    link_blueprint = "https://www.screener.in/company/"
    
    stock_list = ['3MINDIA', 'AUBANK', 'COLPAL', 'GLAND', 'LUXIND', 'WHIRLPOOL']
    
    if stock_name in stock_list:
        actual_link = link_blueprint + stock_name.upper()
    else:
        actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"

    def get_list(actual_link):
        page = requests.get(actual_link)
        link_soup = BeautifulSoup(page.content, 'html.parser')

        test_list = link_soup.find(id="profit-loss").text.replace("  ", "").split('\n')

        while("" in test_list) :
            test_list.remove("")
        while("\r" in test_list) :
            test_list.remove("\r")        
        while(" \r" in test_list) :
            test_list.remove(" \r")

        return test_list

    test_list = get_list(actual_link)

    #print(len(test_list))
    n = len(test_list) 

    if n < 100:
        actual_link = link_blueprint + stock_name.upper() 
        test_list = get_list(actual_link)
        #print(len(test_list))

    q = 0

    if n > 162:
        p = 8
        q = n-162
    elif n > 120 and n <= 162:
        p = 8
    elif n < 120 and n > 100:
        p = 7   

    else:
        p = 6

    
    #print(test_list)
    if 'TTM' in test_list:
        end_year = 'TTM'
    else:
        end_year = 'Mar 2023'
        df_yoy = pd.DataFrame()      
        #print(test_list)
        s = test_list.index(end_year) +1 - test_list.index('Mar 2017')

        df_yoy['Year'] = test_list[test_list.index('Mar 2017'):test_list.index(end_year)+1]
        df_yoy['Sales'] = test_list[test_list.index('Expenses\xa0+')-s:test_list.index('Expenses\xa0+')]
        df_yoy['Profit'] = test_list[test_list.index('Tax %')-s:test_list.index('Tax %')]

        df_yoy.set_index('Year', inplace=True)
        df_yoy = df_yoy.T
        df_yoy['Stock'] = stock_name.upper() + '.NS'
        #df_yoy
        df_latest_analysis = pd.concat([df_latest_analysis, df_yoy])
    
df_latest_analysis 

ADANIPORTS
ASTRAZEN
EQUITAS
IBREALEST
PVRINOX
RAJESHEXPO
SANOFI
SUZLON
TORNTPHARM
VGUARD


Year,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Stock
Sales,"8,439","11,323","10,925","11,873","12,550","15,934","20,852",ADANIPORTS.NS
Profit,"4,179","5,234","5,126","4,244","6,292","5,541","5,489",ADANIPORTS.NS
Sales,544,571,728,832,814,806,"1,003",ASTRAZEN.NS
Profit,36,44,73,114,127,83,134,ASTRAZEN.NS
Sales,"2,320","4,503","4,944","3,271","1,521","1,445",587,IBREALEST.NS
Profit,539,"2,694",844,378,75,-27,-563,IBREALEST.NS
Sales,"2,119","2,334","3,086","3,414",280,"1,331","3,751",PVRINOX.NS
Profit,153,194,299,90,-939,-681,-209,PVRINOX.NS
Sales,"242,132","187,686","175,763","195,600","258,306","243,128","339,690",RAJESHEXPO.NS
Profit,"1,303","1,342","1,346","1,261",894,"1,038","1,478",RAJESHEXPO.NS


In [4]:
print('Stocks updated:', df_latest_analysis.shape[0]/2)
if df_latest_analysis.shape[0] > 0:
  lastestq_stocks = list(df_latest_analysis['Stock'].unique())
  len(lastestq_stocks)

Stocks updated: 8.0


In [5]:
if df_latest_analysis.shape[0] > 0:
  #df_analysis.columns
  df_latest_analysis = df_latest_analysis.reset_index()
  df_latest_analysis.rename({'Mar 2023': 'Latest', 'index': 'Type'}, axis=1, inplace=True)
  cols = [ 'Mar 2017','Mar 2018', 'Mar 2019', 'Mar 2020', 'Mar 2021', 'Mar 2022', 'Latest']
  #df_analysis[cols] = df_analysis[cols].apply(pd.to_numeric)
  for col in cols:
      df_latest_analysis[col] = df_latest_analysis[col].str.replace(',', '').astype(float)

In [6]:
if df_latest_analysis.shape[0] > 0:
  df_latest_analysis['Max'] = df_latest_analysis.max(axis=1)
  df_latest_analysis['Diff%'] = round((df_latest_analysis['Max'] - df_latest_analysis['Latest'])*100/df_latest_analysis['Max'],2)

<ipython-input-6-ff4f08accfa3>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_latest_analysis['Max'] = df_latest_analysis.max(axis=1)


In [7]:
if df_latest_analysis.shape[0] > 0:
  df_master_analysis = pd.read_csv('/content/drive/My Drive/data/stocks/myProspects-ATH-TTM.csv')
  df_master_analysis.columns = df_latest_analysis.columns
  df_master_analysis = df_master_analysis[~df_master_analysis['Stock'].isin(lastestq_stocks)]
  df_master_analysis = pd.concat([df_master_analysis, df_latest_analysis])
  df_master_analysis.to_csv('/content/drive/My Drive/data/stocks/myProspects-ATH-TTM.csv', index=False)

In [8]:
#df_latest_analysis.to_csv('/content/drive/My Drive/data/stocks/Nifty100-ATH-TTM.csv', index=False)

In [9]:
if df_latest_analysis.shape[0] > 0:
  df_tmp = df_latest_analysis[df_latest_analysis['Diff%'] <= 5]
  tmp = pd.DataFrame(df_tmp['Stock'].value_counts())
  len(tmp[tmp['Stock']>1].index)
  star_stocks = list(tmp[tmp['Stock']>1].index)
  print(len(star_stocks))

3


In [10]:
mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')
mypf.loc[mypf['Stock'].isin(star_stocks), 'StarStock'] = 1
mypf.loc[mypf['Stock'].isin(lastestq_stocks), 'LatestQtr'] = 1
mypf.to_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv', index=False)